# Notebook de exploração geral dos dados

## 0. Setup

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 1. Carregar os dados

In [4]:
dados = pd.read_csv('../data/raw/aug_train.csv')

dados.head(3)

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0


In [5]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevent_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  target                  19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

## 2. Manuseio (manipulação) dos dados

In [ ]:
# 1ª mudança: transformar a variável target em "0": não quer trocar e "1": quer trocar

In [12]:
# dados['target'].apply(lambda x: valor_verdadeiro if condicao else 0)

dados['target_label'] = dados['target'].apply(lambda x: 'Quer trocar' if x == 1 else 'Não quer trocar')

In [13]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevent_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  target                  19158 non-null  float64
 14  target_label            19158 non-null

## 3. Análise dos dados

### 3.1. Análise das variável qualitativas

**City:**

- 23% das pessoas moram na cidade 103
- 56% das pessoas moram em uma das 5 principais cidades

**Gender:**

- 69% das pessoas são do gênero masculino
- 23% das pessoas não declaram o seu gênero
- Apenas 6% das pessoas que responderam o formulário são do gênero feminino 
- 1% das pessoas possuem outro tipo de gênero, que não foi listado no formulário

**relevent_experience:**

- A grande maioria (72%) das pessoas tem experiência relevante

**enrolled_university:**

- 72% das pessoas respondentes não estão fazendo faculdade no momento
- 2% das pessoas não responderam o questionário

**education_level:**

- 60% das pessoas têm graduação
- 22% das pessoas têm mestrado
- 10,5% das pessoas têm o ensino médio “americano”
- 2,4% das pessoas têm doutorado

**major_discipline**

- 75% das pessoas respondentes são da área de Exatas (Science, Technology, Engineering, and Math)
- 14% não responderam o questionário

**experience**

- grande parte (17%) das pessoas possuem 20 ou mais anos de experiência
- verificar se o percentual de cada uma das demais categorias são próximos (gráfico)

**company_size**

- 30% das pessoas não responderam essa perguntas
    - hipótese: as pessoas não responderam por opção ou não possuem trabalho
- temos uma quantidade “próxima” de pessoas que trabalham em empresas pequenas, médias e grandes
    - ideia de feature: agrupar os valores de company_size em PP, P, M e G

**company_type**

- A maioria (51%) das pessoas que responderam o questionário trabalham em empresas privadas
- 32% das pessoas não responderam
    - ideia de feature: agrupar os valores de company_type relacionadas a startups

**last_new_job (Diferença em anos do seu último trabalho para o atual)**

- 41% das pessoas tem 1 ano de diferença entre o seu último trabalho e o  atual
- 12% das pessoas nunca ficaram sem trabalho
- 2 anos e 4 anos ou mais na diferença entre o seu último trabalho e o atual, correspondem a 32% das pessoas

In [41]:
lista_var_qual = list(dados.select_dtypes(include = [object]).columns)

In [42]:
for i in lista_var_qual:
    
    print(f'\n----- Variável utilizada: {i} \n')
    
    a = dados[i].value_counts(dropna = False).head(5)
    b = dados[i].value_counts(dropna = False, normalize = True).head(5) * 100
    
    
    c = pd.concat([a, b], axis = 1, keys = ['contagem', '%'])
    
    c['% acc'] = c['%'].cumsum()
    
    display(c)


----- Variável utilizada: city 



,contagem,%,% acc
city_103,4355,22.732018,22.732018
city_21,2702,14.103769,36.835787
city_16,1533,8.001879,44.837666
city_114,1336,6.973588,51.811254
city_160,845,4.410690,56.221944



----- Variável utilizada: gender 



,contagem,%,% acc
Male,13221,69.010335,69.010335
NaN,4508,23.530640,92.540975
Female,1238,6.462052,99.003027
Other,191,0.996973,100.000000



----- Variável utilizada: relevent_experience 



...

,contagem,%,% acc
Pvt Ltd,9817,51.242301,51.242301
NaN,6140,32.049274,83.291575
Funded Startup,1001,5.224971,88.516547
Public Sector,955,4.984863,93.501409
Early Stage Startup,603,3.147510,96.648920



----- Variável utilizada: last_new_job 



,contagem,%,% acc
1,8040,41.966802,41.966802
>4,3290,17.172983,59.139785
2,2900,15.137279,74.277064
never,2452,12.798831,87.075895
4,1029,5.371124,92.447020



----- Variável utilizada: target_label 



,contagem,%,% acc
Não quer trocar,14381,75.065247,75.065247
Quer trocar,4777,24.934753,100.000000
